# HW5

For this homework, we are going to work with [*Indoor User Movement Prediction from RSS data*](https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data) dataset from UCI.  The homework is due Friday, December 21st midnight. 

## Task 1

Download the dataset and unzip it in under a subdirectory of `data` folder named `rss_data`.

The files we are interested is in the subfolder `dataset`.  Each of these files whose names that start with `MovementAAL_RSS_` contain data collected from indivuduals. Each of these files represent a single data point.  There are 314 of these files, and hence, you have 314 data points.  Each file has 4 columns but the number of rows change from file to file.  

There is also a file named `MovementALL_target.csv` in that folder. This file tells us the class each of these measurements are assigned. Some of these measurements are labelled with +1 and some are labelled with -1.

## Task 2

Construct a SVM model that separates +1 labelled data points from -1 data points.  You must first solve the problem that these datapoints do not have the same number of rows even though they all have the same number of columns. 

## Task 3

Using [Keras](https://keras.io/getting-started/sequential-model-guide/) write a neural network model that separates +1 labelled data points from -1 data points.

## Notes

1. You must document each step of your tasks: what are you doing, why are you doing it, what problems you encountered and how you solved it.  All of these must be explained and documented.  Solutions without sufficient documentations will be penalized accordingly. 50% of your points will come from your code, while the other 50% will come from your explanations.

1. You can use MS Excel to inspect the files, but loading them up to python using pandas and inspecting them there under jupyter is easier.

3. Put the data in a separate subfolder of your `data` folder and rename it `rss_data`. I'll take points off if the data is not saved under the correct place.

1. For both of Task 2 and Task 3, you must split your data into a train and test set, and then evaluate the accuracy of your model on the test set.



## Task2: SVM

In [1]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn import neighbors
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from numpy import nan as Nan

import matplotlib.pyplot as plt

In [2]:
df_target = pd.read_csv('C:/Users/azad/Desktop/Data Analysis in Fund. Sciences/github.com/data/rss_data/MovementAAL_target.csv', 
                             names=('sequence_ID', 'class_label'), 
                             skiprows=(1))

In [3]:
def get_dataframe(seq_id):
    return pd.read_csv('C:/Users/azad/Desktop/Data Analysis in Fund. Sciences/github.com/data/rss_data/MovementAAL_RSS_%s.csv' % seq_id, 
                           names=('RSS_anchor1', 'RSS_anchor2','RSS_anchor3', 'RSS_anchor4'), 
                           skiprows=(1))

In [4]:
def create_data(df):
    frames = []
    target = []
    for idx, row in df.iterrows():
        data = get_dataframe(row['sequence_ID'])
        frames.append(data)
        arr = [row['class_label']] * len(data)
        target += arr
        
    return frames, target

data_test, target_test = create_data(df_target)

In [5]:
X = data_test
y = target_test

for i in range(0,314):
    length = len(X[i])
    length1 = 129 - length
    for j in range(0,length1):
        X[i] = X[i].append(pd.Series([np.nan]), ignore_index = True)
    X[i] = X[i].iloc[:, :-1]

In [6]:
total = [None]*4
xs = np.zeros((314,4))

for j in range(0,314):
    for i in range(0,4):
        if(i==0):
            total[0] = np.nansum(X[j].loc[:,'RSS_anchor1'])
        elif(i==1):
            total[1] = np.nansum(X[j].loc[:,'RSS_anchor2'])
        elif(i==2):
            total[2] = np.nansum(X[j].loc[:,'RSS_anchor3'])
        else:
            total[3] = np.nansum(X[j].iloc[:,3:4])
    xs[j] = total

In [7]:
df_target = df_target.drop('sequence_ID', 1)

In [16]:
ys = df_target

classifier = svm.SVC(kernel='linear')

train_xs, test_xs, train_ys, test_ys = train_test_split(xs,ys,test_size=0.25)
classifier.fit(train_xs,train_ys)

predicted_ys = classifier.predict(test_xs)
confusion_matrix(test_ys,predicted_ys)

C:\Users\azad\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[24, 16],
       [12, 27]], dtype=int64)

In [17]:
accuracy_score(test_ys,predicted_ys)

0.6455696202531646

## Task3: Neural Network with Keras

In [30]:
from urllib.request import urlopen
from xmltodict import parse
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import Normalizer

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D
from keras.optimizers import Adam, Adamax, RMSprop, SGD

import keras.backend as K

C:\Users\azad\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
train_xs, test_xs, train_ys, test_ys = train_test_split(xs,ys,test_size=0.25)

In [53]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_xs, train_ys, epochs=40, batch_size=32)



Epoch 1/40
235/235 [==============================] - 0s 833us/step - loss: 6.7459 - acc: 0.4894
Epoch 2/40
235/235 [==============================] - 0s 51us/step - loss: 4.6031 - acc: 0.4809
Epoch 3/40
235/235 [==============================] - 0s 42us/step - loss: 3.0188 - acc: 0.4553
Epoch 4/40
235/235 [==============================] - 0s 47us/step - loss: 1.7699 - acc: 0.3872
Epoch 5/40
235/235 [==============================] - 0s 47us/step - loss: 0.9326 - acc: 0.2809
Epoch 6/40
235/235 [==============================] - 0s 38us/step - loss: 0.4321 - acc: 0.2213
Epoch 7/40
235/235 [==============================] - 0s 42us/step - loss: 0.1122 - acc: 0.1660
Epoch 8/40
235/235 [==============================] - 0s 42us/step - loss: -0.1412 - acc: 0.1489
Epoch 9/40
235/235 [==============================] - 0s 38us/step - loss: -0.3912 - acc: 0.1277
Epoch 10/40
235/235 [==============================] - 0s 38us/step - loss: -0.6104 - acc: 0.1149
Epoch 11/40
235/235 [==============